## Spatial Data Science (GIS6307/GEO4930)

---

# Week 4: Vector and Raster Model


<br>
Instructor: Yi Qiang (qiangy@usf.edu)<br>

---

## 1. Install GeoPandas and Rasterio
Please follow [this instruction](https://github.com/qiang-yi/spatial_data_science/blob/main/other/new_env.ipynb) to set up an new environment and install GeoPandas and Rasterio. After the installation, please close this notebook and launch Jupyter Notebook from the "geo" environment to work on this lab.

In [ ]:
import geopandas as gpd
import rasterio as rio

## 2. Vector Data

### 2.1 Importing a shapefile to geodataframe

Create a folder 'other' in the folder of your jupyter notebook. Then, download the zip file from this [link](https://usf.box.com/s/od9g9cnot83ymqw0y375i4xestqv4lsi). 

The zip file contains a shapefile 'conus.shp', which consists of multiple files with the same file name but different suffixes. These files store different information of the vector data.

First, print files in the "other" folder. All files with a name "conus" are components of the shapefile.

In [ ]:
import glob
glob.glob("other/*")

Read the shapefile into a geodataframe, and preview the geodataframe.

In [ ]:
us = gpd.read_file("other/conus.shp") 

us.head()

You may notice that the geodataframe is actually a Pandas DataFrame + a column of geometry . The geometry contains coordinates (latitude and longitude) of polygons/multipolygons.

Plot the shapefile in a map

In [ ]:
us.plot()

If you feel the figure is too small, you can change the default plot size to make the map larger

In [ ]:
# import matplotlib
import matplotlib.pyplot as plt

# change the default figure size to 10 inches by 10 inches
plt.rcParams['figure.figsize'] = [10, 10]

# plot the shapefile again
us.plot()

### 2.2 Project the geometries

You may feel the Contiguous U.S. is a bit strange, as it is flattened than it is appeared in most U.S. maps.

You can use `.crs` attribute to check the projection of the data

In [ ]:
us.crs

The printed information shows that the shapefile is using a geographic coordinate system (latitude and longitude). The `EPSG:4269` is the unique ID of the coordinate system. There is no projection, meaning the geometries showing above are actually projected using latitude and longitude, which is wrong!

We can use the `to_crs` function to project the geometries to the Albers Equal Area Conic projection (EPSG: 5070), which is the most common projection for a U.S. map.

In [ ]:
# Project geodataframe to the Albers equal area conic projection, and store the projected geodataframe in us2
us2 = us.to_crs(5070)

# preview us2
us2.head()

You may noticed that the coordinates in the geometry columns have changed to meters, meaning that the geometries are projected to the Albers equal area conic projection, using meter as the unit.

Next, plot the geodataframe. You may have observed the difference compared to the unprojected one. The projected geometries look similar to official U.S. maps.

Other than the Albers projection, you can project the geodataframe to any projection using the EPSG code.

In [ ]:
us2.plot()

### 2.3 Creating choropleth maps

You can easily create a choropleth map using values in the 'Population' column. The argument `cmap` defines the color scheme (Orange to Red). You can find more colors from [here](https://matplotlib.org/stable/tutorials/colors/colormaps.html)

In [ ]:
# Create a choropleth map of state population 

us2.plot(column ='Population',cmap='OrRd')

Larger states naturally have more people (larger population). That's why California, Texas and Florida stand out in the above map. But you may not feel lots of people when you actually live there, especially in some areas in Texas.

Instead, population density indicates how "crowded" a state is. The population density of a state is the ratio of population ("Population") to land area of the state ("ALAND"). 

### Quiz 1: Please calculate population density of the states and store it in a new column "Pop_density".

In [ ]:
# Population density = Population/ALAND
us2["Pop_density"] = ...

# Preview the first five rows of us2
us2.head()

### Quiz 2: Please create a choropleth map to show population density in the states.

Please refer to [this website](https://geopandas.org/en/stable/docs/user_guide/mapping.html) to learn more about the plot function. 

You may need to use "fisher_jenks" as the scheme to show more color gradient in the map.

In [ ]:
# Create a choropleth map of population density in states
us2.plot(column ='Pop_density',cmap='OrRd', scheme='fisher_jenks')

### 2.4 Query in GeoDataFrame

A GeoDataFrame is an extension of a dataframe with geometries, meaning that you can use functions in pandas to manipulate GeoDataFrame. For example, you can do queries in a GeoDataFrame as you do it in a DataFrame.

First, let's select Florida from the GeoDataFrame "us2" and show it in a plot.

In [ ]:
# Select the State of Florida and store it in fl
fl = us2[...]

# Plot "fl"
fl.plot()

The boundary of Florida is a bit tilted to the left. This is because the Albers Equal Area Conic projection is suitable for mapping the entire U.S., but is not suitable for local areas.

### Quiz 3: Please project "fl" to the UTM 17N projection (EPSG: 32617), which is a projection that better fits Florida. Then plot the reprojected "fl".

In [ ]:
# Repreject fl to EPSG:32617
fl = ...

# Plot fl again
fl.plot()

### Quiz 4: Please select states where population density is greater than Florida and plot the selected states in a map.

In [ ]:
# get population density of Florida
fl_den = ...

# plot states where population density is larger than Florida
Hi_pop_states =...

# Plot selected states 
Hi_pop_states.plot()

## 3. Raster Data

### 3.1 Read a raster

Raster data is a georeferenced 2D array. The Rasterio library provides a suite of functions to read and manipulate raster data. Next, you will use Rasterio to read and analyze a Digital Elevation Model (DEM) of Hillsborough County. The DEM is a raster where pixel values are elevation (meter) above the sea level.

In [ ]:
# import rasterio and rasterio.plot
import rasterio as rio
import rasterio.plot as rio_pl

# import numpy
import numpy as np

Read the raster file.

In [ ]:
# read the raster data in the other folder
dem = rio.open("other/hill_dem.tif")

Plot the raster in a map

In [ ]:
rio_pl.show(dem)

Print the metadata of the raster.

In [ ]:
dem.profile

The printed profile includes the following information:
- driver: format of the raster file
- dtype: data type of the raster
- nodata: coding of no-data pixels
- width: number of pixels in width
- height: number of pixels in height
- count: number of band
- crs: projection
- transform: Affine(pixel_width, rotation, top_left_x, rotation, -pixel_height, top_left_y)
- blockxsize, blockysize & tiled: information about interal tiling
- interleave: methods for encoding image

As a raster is essentially a georeferenced 2D array, we can get a numpy array from the raster data.

In [ ]:
# Read Band 1 (the only band) of the raster
array = dem.read(1)

# Print the type of the array
print(type(array))

# print the shape of the array
print(array.shape)

### 3.2 Print the statistics of the raster

Print maximum, minimum and mean elevation of the DEM

In [ ]:
print("Maximum elevation: "+ ...)
print("Minimum elevation: "+ ...)
print("Mean elevation: "+ ...)

Do you think the statistics printed above make sense? Is there really a place in Hillsborough County is 128m (420 feet) high?

As shown in the profile, no-data pixels in the raster are coded as 128. So the maximum values are actually the no-data pixels. To perform numerical analysis correctly, we need to mask out the pixels with a value 128. We will convert all no-data pixels to `numpy.ma.masked`

In [ ]:
array[array==128] = np.ma.masked

Print maximum, minimum and mean elevation of the DEM again

In [ ]:
print("Maximum elevation: "+ str(array.max()))
print("Minimum elevation: "+ str(array.min()))
print("Mean elevation: "+ str(array.mean()))

### 3.3 Analyze the raster

Next, we create a map to show only areas where the elevation is equal to or greater than 30m (100 feet). To do so, you need to mask the pixels where elevation is smaller than 30m.

In [ ]:
# Mask pixels where elevation is <30
array[array<30] = np.ma.masked

# Display the unmasked pixels (elevation >= 30)
rio_pl.show(array)

Low-lying coastal areas are at a high-risk of flooding and storm surge. Next, we will select areas where elevation is equal to or less than 1 meter and plot the areas in a map.

### Quiz 5 (2pt): Please complete the following code to display areas where elevation is equal to or less than 1 meter.

In [ ]:
# Read the numpy array from Band 1 of the DEM
array = dem.read(1)

# Mask pixels where elevation is greater than 1
array[array>1] = np.ma.masked

# Displayed the unmasked pixels
rio_pl.show(array,cmap='OrRd')